In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import re
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import TfidfModel, LdaMulticore
from wordcloud import WordCloud
from datetime import datetime
from collections import Counter
import seaborn as sns
from textblob import TextBlob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/IDX_FILE_TEXT_DAT.csv')

In [ ]:
def clean(x):
  for i in tqdm(range(len(df))):
    try:
      x = re.sub(r"[^a-zA-Z0-9 ]", " ", df['text'][i])
      x = x.lower()
      x = re.sub(' {2,}', ' ', x)
      df.loc[i,'text'] = x
    except:
      continue


In [ ]:
clean(df)

In [ ]:
my_file = open("/content/drive/MyDrive/StopWords_Generic.txt", "r")
content = my_file.read()
content_list_1 = content.split("\n")
my_file.close()
content_list_1 = [x.lower() for x in content_list_1]

my_file = open("/content/drive/MyDrive/StopWords_GenericLong.txt", "r")
content = my_file.read()
content_list_2 = content.split("\n")
my_file.close()
content_list_2 = [x.lower() for x in content_list_2]

my_file = open("/content/drive/MyDrive/StopWords_DatesandNumbers.txt", "r")
content = my_file.read()
content_list_3 = content.split("\n")
my_file.close()
content_list_3 = [x.lower() for x in content_list_3]



In [ ]:
content_list_1.extend(content_list_2)

In [ ]:
content_list_1.extend(content_list_3)

In [ ]:
extra_stopwords = content_list_1

In [ ]:

import nltk
nltk.download('stopwords')


def lemmetize(words):
    lemmatized_words = [WordNetLemmatizer().lemmatize(w.lower()) for w in words]
    return lemmatized_words


stop_words = set(stopwords.words('english'))
fin_stop_words = ("million","including","billion","december","january")
stop_words.update(extra_stopwords)

# removing stop words, numbers , removing punctuations and special characters and spaces
def remove_stopwords(words):
    filtered = [re.sub(r'[^\w\s]','',w) for w in words if not re.sub(r'[^\w\s]','',w) in stop_words and  not re.sub(r'[^\w\s]','',w).isnumeric() and not re.search('^\s*[0-9]',re.sub(r'[^\w\s]','',w)) and len(re.sub(r'[^\w\s]','',w)) > 3  ]
    return filtered


from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

   
#ps = PorterStemmer() 
ps = SnowballStemmer("english") 

def stem_words(words):
    stemmed = [ps.stem(w) for w in words]
    return stemmed


In [ ]:
df['lists'] = [list() for x in range(len(df.index))]

In [ ]:
df_1 = df[:12500]
df_2 = df[12500:25000]
df_3 = df[25000:]

In [ ]:
del df
df = None

In [ ]:
df_1 = df_1.reset_index(drop=True)
df_2 = df_2.reset_index(drop=True)
df_3 = df_3.reset_index(drop=True)

In [ ]:
import nltk
nltk.download('punkt')

for i in tqdm(range(len(df_1))):
  try:
    lemmatized_words = [WordNetLemmatizer().lemmatize(w.lower()) for w in nltk.word_tokenize(df_1['text'][i])]
    x = remove_stopwords(lemmatized_words)

    df_1.loc[i,'lists'].append(x)
    df_1.loc[i, 'text'] = np.nan
  except:
    continue

df_1.to_csv('/content/drive/MyDrive/Colab Notebooks/DATA/DATASET 1 SIMILARITY.csv')

from google.colab import files

df_1.to_csv('df_1.csv')
files.download('df_1.csv')

In [ ]:
import nltk
nltk.download('punkt')

for i in tqdm(range(len(df_2))):
  try:
    lemmatized_words = [WordNetLemmatizer().lemmatize(w.lower()) for w in nltk.word_tokenize(df_2['text'][i])]
    x = remove_stopwords(lemmatized_words)

    df_2.loc[i,'lists'].append(x)
    df_2.loc[i, 'text'] = np.nan
  except:
    continue

df_2.to_csv('/content/drive/MyDrive/Colab Notebooks/DATA/DATASET 2 SIMILARITY.csv')

from google.colab import files

df_2.to_csv('df_2.csv')
files.download('df_2.csv')

In [ ]:
import nltk
nltk.download('punkt')

for i in tqdm(range(len(df_3))):
  try:
    lemmatized_words = [WordNetLemmatizer().lemmatize(w.lower()) for w in nltk.word_tokenize(df_3['text'][i])]
    x = remove_stopwords(lemmatized_words)

    df_3.loc[i,'lists'].append(x)
    df_3.loc[i, 'text'] = np.nan
  except:
    continue

df_3.to_csv('/content/drive/MyDrive/Colab Notebooks/DATA/DATASET 2 SIMILARITY.csv')

from google.colab import files

df_3.to_csv('df_3.csv')
files.download('df_3.csv')

In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/DOC SIMILARITY/DATASET 1 SIMILARITY.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/DOC SIMILARITY/DATASET 2 SIMILARITY.csv')
df_3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/DOC SIMILARITY/DATASET 3 SIMILARITY.csv')

In [ ]:
df = pd.concat([df_1,df_2],ignore_index = True)

df = pd.concat([df,df_3],ignore_index = True)

In [ ]:
df.sort_values(by=['permno','date'])

df = df.reset_index(drop = True)             

df['counter'] = df.groupby(['permno']).cumcount()+1


In [ ]:
from collections import Counter

for i in tqdm(range(len(df))):
  l1 =[];l2 =[]
  try:
    if df['counter'][i] == 1:
      df.loc[i,'cosine'] = 0
      df.loc[i,'jaccard'] = 0
      df.loc[i, 'text'] = np.nan
    else:
        set_1 = set(df['lists'][i-1].split(','))
        set_2 = set(df['lists'][i].split(','))

        rvector = set_1.union(set_2) 

        counter_1 = Counter(df['lists'][i-1].split(','))
        counter_2 = Counter(df['lists'][i].split(','))



        for w in rvector:
          if w in set_1: l1.append(counter_1[w]) 
          else: l1.append(0)
          if w in set_2: l2.append(counter_2[w])
          else: l2.append(0)

        c = 0

        for g in range(len(rvector)):
          c += l1[g]*l2[g]

        cosine = c / float(((sum([pow(item, 2) for item in l1])**0.5) * (sum([pow(item, 2) for item in l2])**0.5)))

        df.loc[i,'cosine'] = cosine

        intersection = set_1.intersection(set_2)

        jaccard = float(len(intersection)) / len(rvector)

        df.loc[i,'jaccard'] = jaccard

        df.loc[i, 'text'] = np.nan

  except:
    continue

In [ ]:
df = df.drop(['lists','text','counter','Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'], 1)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/DATA/SIMILARITY RESULTS FINAL.csv')